In [ ]:
# List all the files in the training data directory
import os

import numpy as np
import pandas as pd
import respiration.utils as utils

# Get the list of files in the training data directory
training_data_dir = utils.dir_path('models', 'rhythm_former_v2')

configs = []

for model_dir in os.listdir(training_data_dir):
    manifest_path = utils.join_paths(training_data_dir, model_dir, 'manifest.json')

    # Check if the manifest file exists
    if not os.path.exists(manifest_path) or model_dir == "20240905_094727":
        continue

    # Load the manifest file
    manifest = utils.read_json(manifest_path)

    setting = manifest['testing_scenarios'][0][1]

    # Get all column with a '_weight' suffix
    components = [
        key for key, value in manifest['loss_fn_config'].items() if (key.endswith('_weight') and value != 0)
    ]

    # Remove the '_weight' suffix from the keys
    components = [component[:-7] for component in components]

    # Add the metadata to the records list
    configs.append({
        'model': 'RF_' + manifest['timestamp'],
        'setting': setting,
        'image_dimension': manifest['image_size'][0],
        'best_epoch': manifest['models'][-1]['epoch'],
        'frequency_weight': manifest['loss_fn_config']['frequency_weight'],
        'mse_weight': manifest['loss_fn_config']['mse_weight'],
        'norm_weight': manifest['loss_fn_config']['norm_weight'],
        'pearson_weight': manifest['loss_fn_config']['pearson_weight'],
        'spectral_convergence_weight': manifest['loss_fn_config']['spectral_convergence_weight'],
        'spectral_magnitude_weight': manifest['loss_fn_config']['spectral_magnitude_weight'],
        'split': manifest['split'],
        'components': sorted(components),
    })

# Create a DataFrame from the records list
configs = pd.DataFrame(configs)

# Sort the DataFrame by the model
configs = configs.sort_values(by='model')

configs

In [ ]:
analysis_dir = utils.dir_path('outputs', 'analysis')

frequencies_file = utils.join_paths(analysis_dir, 'frequencies.csv')
frequencies = pd.read_csv(frequencies_file)

metrics_file = utils.join_paths(analysis_dir, 'metrics.csv')
metrics = pd.read_csv(metrics_file)

metrics_avg_file = utils.join_paths(analysis_dir, 'metrics_average.csv')
metrics_average = pd.read_csv(metrics_avg_file)

In [ ]:
figure_dir = utils.dir_path('outputs', 'figures', mkdir=True)

In [ ]:
# Plot the MAE and PCC for the psd method
import seaborn as sns
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(12, 6))

# Only keep the models that start with "RF_"
points = metrics_average[metrics_average['model'].str.startswith('RF_')]

# Scatter the MAE and PCC for the different models
sns.scatterplot(
    data=points,
    x='mae',
    y='pcc',
    s=250,
    style='model',
    hue='model',
)

plt.xlabel('MAE (BPM)')
plt.ylabel('Correlation')
plt.title('MAE and Pearson Correlation for the different models')
plt.tight_layout()

# Set the dimensions of the plot
# plt.xlim(0, 8)
# plt.ylim(0, 1)

# Place the legend outside the plot
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.show()

In [ ]:
# Merge the points and configs DataFrames
data = pd.merge(configs, points, on='model')
data.to_csv(utils.join_paths(analysis_dir, 'loss_function.csv'), index=False)
data

## Analyse Loss Components

In [ ]:
from scipy import stats

loss_components = {
    'frequency_weight',
    'mse_weight',
    'norm_weight',
    'pearson_weight',
    'spectral_convergence_weight',
    'spectral_magnitude_weight',
    # 'image_dimension',
}

natural_setting = data[(data['setting'] == '101_natural_lighting') & (data['image_dimension'] == 128)]

loss_influence = []

for component in loss_components:
    group = natural_setting.groupby(component)

    models_0 = group.get_group(0)['model']
    models_1 = group.get_group(1)['model']

    frequencies_0 = frequencies[frequencies['model'].isin(models_0)]
    frequencies_1 = frequencies[frequencies['model'].isin(models_1)]
    
    error_0 = np.abs(frequencies_0['prediction'] - frequencies_0['ground_truth'])
    error_1 = np.abs(frequencies_1['prediction'] - frequencies_1['ground_truth'])

    # Perform independent two-sample t-test
    t_stat, p_value = stats.ttest_ind(error_0, error_1)

    loss_influence.append({
        'component': component[:-7],
        't_stat': round(t_stat, 3),
        'p_value': round(p_value, 3),
        'mae_0': round(error_0.mean() * 60, 1),
        'mae_1': round(error_1.mean() * 60, 1),
        'models_0': len(models_0),
        'models_1': len(models_1),
        'points_0': len(frequencies_0),
        'points_1': len(frequencies_1),
    })

loss_influence = pd.DataFrame(loss_influence)
loss_influence.to_csv(utils.join_paths(analysis_dir, 'loss_influence.csv'), index=False)
loss_influence

## Analyse Scenario Influence

In [ ]:
# Get all models that are trained on the '303_normalized_face' setting
normalized_face = configs[configs['setting'] == '303_normalized_face']

face_models = []
normal_models = []

for _, row in normalized_face.iterrows():
    face_models.append(row['model'])

    # Find all models with the same components
    data = configs[(configs['setting'] != '303_normalized_face') &
                   (configs['image_dimension'] == 128)]

    for _, row2 in data.iterrows():
        if row2['components'] == row['components']:
            normal_models.append(row2['model'])

            break

print(face_models)
print(normal_models)

In [ ]:
face_freq = frequencies[frequencies['model'].isin(face_models)]
normal_freq = frequencies[frequencies['model'].isin(normal_models)]

face_error = np.abs(face_freq['prediction'] - face_freq['ground_truth'])
normal_error = np.abs(normal_freq['prediction'] - normal_freq['ground_truth'])

print(f'Face:   {face_error.mean() * 60:.1f} ± {face_error.std() * 60:.1f}')
print(f'Normal: {normal_error.mean() * 60:.1f} ± {normal_error.std() * 60:.1f}')

# Perform independent two-sample t-test
face_normal_t, face_normal_p = stats.ttest_ind(face_error, normal_error)

print(f'T-statistic: {face_normal_t:.3f}, P-value: {face_normal_p:.3f}')